## Import Libraries and Google API Key

In [1]:
# Dependencies
import pandas as pd
import numpy as np
import requests
import json
import gmaps
# Google API Key
from config import gkey
gmaps.configure(api_key=gkey)

In [2]:
# Import wine regions csv file as DataFrame
wine_region_pd = pd.read_csv("thirty_dollars_clean.csv")
wine_region_pd.head()

,id,alcohol,category,country,country_clean,description,designation,price,rating,region,region_clean,subregion,subsubregion,title,url,varietal,vintage,winery
0,51869,17.6,Red,Australia,Australia,"Pours as viscous as motor oil, and has the mos...",Museum Reserve,16,95,Victoria,Melbourne,Victoria,NaN,Benjamin NV Museum Reserve Muscat (Victoria),https://www.winemag.com/buying-guide/benjamin-...,Muscat,NaN,Benjamin
1,210147,13.0,Red,Austria,Austria,"A wave of ripe, juicy black cherry reaches the...",Ried Spitzerberg,26,95,Carnuntum,Vienna,NaN,NaN,Trapl 2015 Ried Spitzerberg Blaufränkisch (Car...,https://www.winemag.com/buying-guide/trapl-201...,Blaufränkisch,2015.0,Trapl
2,152,13.5,White,Austria,Austria,"Salt, pepper and lemon notes own the nose offe...",Ried Gaisberg,29,95,Kamptal,Kamptal,NaN,NaN,Eichinger 2017 Ried Gaisberg Grüner Veltliner ...,https://www.winemag.com/buying-guide/eichinger...,Grüner Veltliner,2017.0,Eichinger
3,77598,13.0,White,Austria,Austria,Clouds of aromatic flower play along with zest...,Gaisberg Reserve,29,95,Kamptal,Kamptal,NaN,NaN,Eichinger 2015 Gaisberg Reserve Riesling (Kamp...,https://www.winemag.com/buying-guide/eichinger...,Riesling,2015.0,Eichinger
4,210134,13.0,White,Austria,Austria,This wine must have spent very little time in ...,Pfaffenberg,20,95,Kremstal,Krems,NaN,NaN,Lesehof Stagård 2016 Pfaffenberg Riesling (Kre...,https://www.winemag.com/buying-guide/lesehof-s...,Riesling,2016.0,Lesehof Stagård


In [3]:
# Add columns for lat, lng 
# Note that we used "" to specify initial entry.
wine_region_pd["Lat"] = ""
wine_region_pd["Lng"] = ""

wine_region_pd.head()

,id,alcohol,category,country,country_clean,description,designation,price,rating,region,region_clean,subregion,subsubregion,title,url,varietal,vintage,winery,Lat,Lng
0,51869,17.6,Red,Australia,Australia,"Pours as viscous as motor oil, and has the mos...",Museum Reserve,16,95,Victoria,Melbourne,Victoria,NaN,Benjamin NV Museum Reserve Muscat (Victoria),https://www.winemag.com/buying-guide/benjamin-...,Muscat,NaN,Benjamin,,
1,210147,13.0,Red,Austria,Austria,"A wave of ripe, juicy black cherry reaches the...",Ried Spitzerberg,26,95,Carnuntum,Vienna,NaN,NaN,Trapl 2015 Ried Spitzerberg Blaufränkisch (Car...,https://www.winemag.com/buying-guide/trapl-201...,Blaufränkisch,2015.0,Trapl,,
2,152,13.5,White,Austria,Austria,"Salt, pepper and lemon notes own the nose offe...",Ried Gaisberg,29,95,Kamptal,Kamptal,NaN,NaN,Eichinger 2017 Ried Gaisberg Grüner Veltliner ...,https://www.winemag.com/buying-guide/eichinger...,Grüner Veltliner,2017.0,Eichinger,,
3,77598,13.0,White,Austria,Austria,Clouds of aromatic flower play along with zest...,Gaisberg Reserve,29,95,Kamptal,Kamptal,NaN,NaN,Eichinger 2015 Gaisberg Reserve Riesling (Kamp...,https://www.winemag.com/buying-guide/eichinger...,Riesling,2015.0,Eichinger,,
4,210134,13.0,White,Austria,Austria,This wine must have spent very little time in ...,Pfaffenberg,20,95,Kremstal,Krems,NaN,NaN,Lesehof Stagård 2016 Pfaffenberg Riesling (Kre...,https://www.winemag.com/buying-guide/lesehof-s...,Riesling,2016.0,Lesehof Stagård,,


In [4]:
# Create a parameters dictionary that will be updated with new city / wine region each iteration
params = {"key": gkey}

# Loop through the cities_pd and run a lat/long search for each city / wine region
for index, row in wine_region_pd.iterrows():
    base_url = "https://maps.googleapis.com/maps/api/geocode/json"

    city = row['region_clean']
    state = row['country_clean']

    # Update address key value
    params['address'] = f"{city},{state}"

    # make request
    cities_lat_lng = requests.get(base_url, params=params)
    
    # print the cities_lat_lng url, avoid doing for public github repos in order to avoid exposing key
    # print(cities_lat_lng.url)
    
    # convert to json
    cities_lat_lng = cities_lat_lng.json()

    wine_region_pd.loc[index, "Lat"] = cities_lat_lng["results"][0]["geometry"]["location"]["lat"]
    wine_region_pd.loc[index, "Lng"] = cities_lat_lng["results"][0]["geometry"]["location"]["lng"]

# Visualize to confirm lat lng appear
wine_region_pd.head()

,id,alcohol,category,country,country_clean,description,designation,price,rating,region,region_clean,subregion,subsubregion,title,url,varietal,vintage,winery,Lat,Lng
0,51869,17.6,Red,Australia,Australia,"Pours as viscous as motor oil, and has the mos...",Museum Reserve,16,95,Victoria,Melbourne,Victoria,NaN,Benjamin NV Museum Reserve Muscat (Victoria),https://www.winemag.com/buying-guide/benjamin-...,Muscat,NaN,Benjamin,-37.8136,144.963
1,210147,13.0,Red,Austria,Austria,"A wave of ripe, juicy black cherry reaches the...",Ried Spitzerberg,26,95,Carnuntum,Vienna,NaN,NaN,Trapl 2015 Ried Spitzerberg Blaufränkisch (Car...,https://www.winemag.com/buying-guide/trapl-201...,Blaufränkisch,2015.0,Trapl,48.2082,16.3738
2,152,13.5,White,Austria,Austria,"Salt, pepper and lemon notes own the nose offe...",Ried Gaisberg,29,95,Kamptal,Kamptal,NaN,NaN,Eichinger 2017 Ried Gaisberg Grüner Veltliner ...,https://www.winemag.com/buying-guide/eichinger...,Grüner Veltliner,2017.0,Eichinger,48.5851,15.4127
3,77598,13.0,White,Austria,Austria,Clouds of aromatic flower play along with zest...,Gaisberg Reserve,29,95,Kamptal,Kamptal,NaN,NaN,Eichinger 2015 Gaisberg Reserve Riesling (Kamp...,https://www.winemag.com/buying-guide/eichinger...,Riesling,2015.0,Eichinger,48.5851,15.4127
4,210134,13.0,White,Austria,Austria,This wine must have spent very little time in ...,Pfaffenberg,20,95,Kremstal,Krems,NaN,NaN,Lesehof Stagård 2016 Pfaffenberg Riesling (Kre...,https://www.winemag.com/buying-guide/lesehof-s...,Riesling,2016.0,Lesehof Stagård,48.41,15.6038


In [ ]:
## Heatmap showing the price of wines under $30
#####  You probably noticed they are all in the 30 to 50 degree Latitude 
Parallels of the continent 

In [5]:
wine_loc=wine_region_pd[["Lat","Lng"]]
price = wine_region_pd["price"].astype(float)

fig = gmaps.figure()
heat_layer=gmaps.heatmap_layer(wine_loc, weights = price, dissipating=False, max_intensity=10, point_radius=1)
fig.add_layer(heat_layer)
heat_layer.gradient = [
    (219, 252, 3, 0.1),
    (3, 244, 252, 0.3),
    (252, 3, 173, 0.6)
]
fig

Figure(layout=FigureLayout(height='420px'))

In [6]:
#Find a Hotel

# Parameters dictionary to update each iteration
params = {
    "radius": 25000,
    "types": "lodging",
    "keyword": "hotel",
    "key": gkey
}

# Use the lat/lng we recovered to identify airports
for index, row in wine_region_pd.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip 
    # any that are missing a data point.
    try:
        wine_region_pd.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
        wine_region_pd.loc[index, "Hotel Address"] = name_address["results"][0]["vicinity"]
        wine_region_pd.loc[index, "Hotel Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [7]:
# Save Data to csv
wine_region_pd.to_csv("Output\Time_To_Sleep.csv")

# Visualize to confirm hotel data appears
wine_region_pd

,id,alcohol,category,country,country_clean,description,designation,price,rating,region,...,title,url,varietal,vintage,winery,Lat,Lng,Hotel Name,Hotel Address,Hotel Rating
0,51869,17.6,Red,Australia,Australia,"Pours as viscous as motor oil, and has the mos...",Museum Reserve,16,95,Victoria,...,Benjamin NV Museum Reserve Muscat (Victoria),https://www.winemag.com/buying-guide/benjamin-...,Muscat,NaN,Benjamin,-37.8136,144.963,The Great Southern Hotel,"44 Spencer St, Melbourne",3.7
1,210147,13.0,Red,Austria,Austria,"A wave of ripe, juicy black cherry reaches the...",Ried Spitzerberg,26,95,Carnuntum,...,Trapl 2015 Ried Spitzerberg Blaufränkisch (Car...,https://www.winemag.com/buying-guide/trapl-201...,Blaufränkisch,2015.0,Trapl,48.2082,16.3738,H+ Hotel Wien,"Liechtensteinstraße 87-89, Wien",4.6
2,152,13.5,White,Austria,Austria,"Salt, pepper and lemon notes own the nose offe...",Ried Gaisberg,29,95,Kamptal,...,Eichinger 2017 Ried Gaisberg Grüner Veltliner ...,https://www.winemag.com/buying-guide/eichinger...,Grüner Veltliner,2017.0,Eichinger,48.5851,15.4127,Hotel Weinquadrat,"Landstraße 238, Weißenkirchen in der Wachau",4.8
3,77598,13.0,White,Austria,Austria,Clouds of aromatic flower play along with zest...,Gaisberg Reserve,29,95,Kamptal,...,Eichinger 2015 Gaisberg Reserve Riesling (Kamp...,https://www.winemag.com/buying-guide/eichinger...,Riesling,2015.0,Eichinger,48.5851,15.4127,Hotel Weinquadrat,"Landstraße 238, Weißenkirchen in der Wachau",4.8
4,210134,13.0,White,Austria,Austria,This wine must have spent very little time in ...,Pfaffenberg,20,95,Kremstal,...,Lesehof Stagård 2016 Pfaffenberg Riesling (Kre...,https://www.winemag.com/buying-guide/lesehof-s...,Riesling,2016.0,Lesehof Stagård,48.41,15.6038,Hotel Weinquadrat,"Landstraße 238, Weißenkirchen in der Wachau",4.8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,18583,NaN,Red,US,Washington,"Dark, tannic and smoky, this has more tarry to...",NaN,30,95,Washington,...,Fielding Hills 2004 Cabernet Sauvignon (Wahluk...,https://www.winemag.com/buying-guide/fielding-...,Cabernet Sauvignon,2004.0,Fielding Hills,46.3238,-117.972,Best Western Plus Dayton Hotel & Suites,"507 E Main St, Dayton",4.6
66,199677,NaN,White,US,Washington,"Among McCrea's Rhône-inspired white wines, thi...",Boushey Vineyard Sirocco Blanc,25,95,Washington,...,McCrea 2008 Boushey Vineyard Sirocco Blanc Whi...,https://www.winemag.com/buying-guide/mccrea-20...,White Blend,2008.0,McCrea,46.3238,-117.972,Best Western Plus Dayton Hotel & Suites,"507 E Main St, Dayton",4.6
67,37687,NaN,Red,US,Washington,Sophisticated winemaking meets gorgeous fruit ...,Stone River Red Wine,28,95,Washington,...,Beresan 2002 Stone River Red Wine Red (Walla W...,https://www.winemag.com/buying-guide/beresan-2...,Red Blend,2002.0,Beresan,46.0646,-118.343,Marcus Whitman Hotel & Conference Center,"6 W Rose St, Walla Walla",4.5
68,131038,14.2,White,US,Washington,"Simply sensational, this estate-grown Viognier...",NaN,30,95,Washington,...,Abeja 2012 Viognier (Walla Walla Valley (WA)),https://www.winemag.com/buying-guide/abeja-201...,Viognier,2012.0,Abeja,46.0646,-118.343,Marcus Whitman Hotel & Conference Center,"6 W Rose St, Walla Walla",4.5


In [8]:
#Find a restaurant

# Parameters dictionary to update each iteration
params = {
    "radius": 25000,
    "types": "restaurant",
    "keyword": "wine",
    "key": gkey
}

# Use the lat/lng we recovered to identify restaurants
for index, row in wine_region_pd.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]

    # change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "restaurant" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
#     print the name_address url, avoid doing for public github repos in order to avoid exposing key
#     print(name_address.url)

    # convert to json
    name_address = name_address.json()
    # print(json.dumps(name_address, indent=4, sort_keys=True))

    # Since some data may be missing we incorporate a try-except to skip 
    # any that are missing a data point.
    try:
        wine_region_pd.loc[index, "Restaurant Name"] = name_address["results"][0]["name"]
        wine_region_pd.loc[index, "Restaurant Address"] = name_address["results"][0]["vicinity"]
        wine_region_pd.loc[index, "Restaurant Rating"] = name_address["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [9]:
# Save Data to csv
wine_region_pd.to_csv("Output\Time_To_Eat.csv")

# Visualize to confirm hotel data appears
wine_region_pd

,id,alcohol,category,country,country_clean,description,designation,price,rating,region,...,vintage,winery,Lat,Lng,Hotel Name,Hotel Address,Hotel Rating,Restaurant Name,Restaurant Address,Restaurant Rating
0,51869,17.6,Red,Australia,Australia,"Pours as viscous as motor oil, and has the mos...",Museum Reserve,16,95,Victoria,...,NaN,Benjamin,-37.8136,144.963,The Great Southern Hotel,"44 Spencer St, Melbourne",3.7,Punch Lane Wine Bar & Restaurant,"43 Little Bourke St, Melbourne",4.5
1,210147,13.0,Red,Austria,Austria,"A wave of ripe, juicy black cherry reaches the...",Ried Spitzerberg,26,95,Carnuntum,...,2015.0,Trapl,48.2082,16.3738,H+ Hotel Wien,"Liechtensteinstraße 87-89, Wien",4.6,Kiang Dine and Wine,"Grünentorgasse 19/2-3, Wien",4.8
2,152,13.5,White,Austria,Austria,"Salt, pepper and lemon notes own the nose offe...",Ried Gaisberg,29,95,Kamptal,...,2017.0,Eichinger,48.5851,15.4127,Hotel Weinquadrat,"Landstraße 238, Weißenkirchen in der Wachau",4.8,Rogl's Keller,"Dürnstein 103 b, Rogl's Keller, Dürnstein",4.7
3,77598,13.0,White,Austria,Austria,Clouds of aromatic flower play along with zest...,Gaisberg Reserve,29,95,Kamptal,...,2015.0,Eichinger,48.5851,15.4127,Hotel Weinquadrat,"Landstraße 238, Weißenkirchen in der Wachau",4.8,Rogl's Keller,"Dürnstein 103 b, Rogl's Keller, Dürnstein",4.7
4,210134,13.0,White,Austria,Austria,This wine must have spent very little time in ...,Pfaffenberg,20,95,Kremstal,...,2016.0,Lesehof Stagård,48.41,15.6038,Hotel Weinquadrat,"Landstraße 238, Weißenkirchen in der Wachau",4.8,Rogl's Keller,"Dürnstein 103 b, Rogl's Keller, Dürnstein",4.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,18583,NaN,Red,US,Washington,"Dark, tannic and smoky, this has more tarry to...",NaN,30,95,Washington,...,2004.0,Fielding Hills,46.3238,-117.972,Best Western Plus Dayton Hotel & Suites,"507 E Main St, Dayton",4.6,Fiesta En Jalisco,"400 W Main St, Dayton",4.2
66,199677,NaN,White,US,Washington,"Among McCrea's Rhône-inspired white wines, thi...",Boushey Vineyard Sirocco Blanc,25,95,Washington,...,2008.0,McCrea,46.3238,-117.972,Best Western Plus Dayton Hotel & Suites,"507 E Main St, Dayton",4.6,Fiesta En Jalisco,"400 W Main St, Dayton",4.2
67,37687,NaN,Red,US,Washington,Sophisticated winemaking meets gorgeous fruit ...,Stone River Red Wine,28,95,Washington,...,2002.0,Beresan,46.0646,-118.343,Marcus Whitman Hotel & Conference Center,"6 W Rose St, Walla Walla",4.5,Whitehouse-Crawford,"55 W Cherry St, Walla Walla",4.5
68,131038,14.2,White,US,Washington,"Simply sensational, this estate-grown Viognier...",NaN,30,95,Washington,...,2012.0,Abeja,46.0646,-118.343,Marcus Whitman Hotel & Conference Center,"6 W Rose St, Walla Walla",4.5,Whitehouse-Crawford,"55 W Cherry St, Walla Walla",4.5
